# Python 2 标准库示例：2.6 struct-二进制数据结构


**目的**: 字节字符串与二进制数据之间的转换。

**Python 版本**: 1.4+

*struct* 模块实现了 Python 数值与用字节字符串表示的 C struct 数据间的转换。

可以采用 *struct* 模块级的函数进行转换工作，也可以先创建一个 *Struct* 类实例，用其方法进行操作。

存储格式指示串和正则表达式类似，也会进行预编译。


# 打包与解包

*struct* 通过 *pack()* 将 Python 数值打包成字节字符串，用 *unpack()* 将字节中解包回 Python 数值。

下面的例子中，存储格式指示串指明了： 1 个整数，1 个 2 字符的字符串，1 个浮点数。

In [1]:
import struct
import binascii

values = (1, 'ab', 2.7)
s = struct.Struct('I 2s f')
packed_data = s.pack(*values)

print 'Original values:', values
print 'Format string:', s.format
print 'Uses:', s.size, 'bytes' # s.size = struct.calcsize(s.format)
print 'Packed values:', binascii.hexlify(packed_data)

Original values: (1, 'ab', 2.7)
Format string: I 2s f
Uses: 12 bytes
Packed values: 0100000061620000cdcc2c40


使用 *unpack()* 进行解包。

In [2]:
import struct
import binascii

packed_data = binascii.unhexlify('0100000061620000cdcc2c40')

s = struct.Struct('I 2s f')
unpacked_data = s.unpack(packed_data)
print 'Unpacked values:', unpacked_data

Unpacked values: (1, 'ab', 2.700000047683716)


注意到上面 *unpack()* 回来后浮点数值的变化。


# 格式字符串

格式字符串是一种用于指定预期数据存储布局的一种机制。它由格式字符组成，而格式字符指定数据的转换类型。

下面是格式字符